<a href="https://colab.research.google.com/github/Aleksandr62aa/AI_ML_DL/blob/main/Detect__Tracking_Counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Detect, count persons an image | Detect, track, count persons an video and DeepSORT**

In [ ]:
# Setup YOLO
!pip install ultralytics
from ultralytics import YOLO
import ultralytics
ultralytics.__version__

**Clone the Github Repo**

In [ ]:
!git clone https://github.com/Aleksandr62aa/Object_Tracking_DeepSORT_YOLOv8.git
%cd Object_Tracking_DeepSORT_YOLOv8

Cloning into 'Object_Tracking_DeepSORT_YOLOv8'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 173 (delta 37), reused 0 (delta 0), pack-reused 97
Receiving objects: 100% (173/173), 58.77 MiB | 35.40 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/Object_Tracking_DeepSORT_YOLOv8


**Detect, count persons an image**

**Object detector YOLO + count persons**

In [ ]:
# Importing libraries
import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

In [ ]:
# class names YOLO
class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
               'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
               'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
               'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
               'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
               'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
               'scissors', 'teddy bear', 'hair drier', 'toothbrush']

unique_ids = set()

In [ ]:
# load a pretrained model
model = YOLO("yolov8n.pt")

# detect objects in image using a model YOLO
results = model("/content/Persons2.jpg", classes=0, conf=0.8, save=True)

# parsing model parameters
result = results[0]
boxes = result.boxes  # boxes object

# Object label (class name)
class_names = [result.names[box.cls.item()] for box in boxes] # box with cls format, (1, N)
# print("Class name -->", class_names)

# Confidence object
confidences = [round(box.conf.item(), 2)  for box in boxes]   # box with conf format, (1, N)
# print("Confidence -->",confidences)

# Coordinates of the object's bounding box
xyxys = boxes.xyxy # box with xyxy format, (N, 4)
xywhs = boxes.xywh # box with xywh format, (N, 4)
# print(xywhs)

# load a image
img = cv2.imread('/content/Persons2.jpg')

for id, (xyxy, confidence, class_name) in enumerate(zip(xyxys, confidences, class_names)):
    x1, y1, x2, y2 = xyxy
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    # Display the frame of an object in the image
    color = (0, 255, 0) # green
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)

    # Display text on an image
    text_color = (0, 0, 0)  # Black color for text
    cv2.putText(img, f"{id}  {confidence}", (int(x1) + 10, int(y1) - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, text_color, 2, cv2.LINE_AA)

    # Add id to the set of unique IDs
    unique_ids.add(id)


# Update the person count based on the number of unique track IDs
person_count = len(unique_ids)

# Display the number of people in the image
text_color = (0, 0, 0)  # Black color for text
cv2.putText(img, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
            1, text_color, 2, cv2.LINE_AA)

# Save the image - object detector and count persons
cv2.imwrite("/content/persons2_.jpg", img)

100%|██████████| 6.23M/6.23M [00:00<00:00, 65.3MB/s]

image 1/1 /content/Persons2.jpg: 448x640 2 persons, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict


True

**Detect, track, count persons an video and DeepSORT**

**Object detector YOLO + DeepSort**

In [ ]:
# Importing libraries DeepSort
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [ ]:
# Input the video path
video_path = '/content/test3.mp4'

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
      print("Error video file")

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Output the video path
output_path = 'output.avi'
out = cv2.VideoWriter(output_path, fourcc, 10, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# parameters
counter, fps, elapsed = 0, 0, 0
# Set color values for red, blue, and green
red_color = (0, 0, 255)  # (B, G, R)
blue_color = (255, 0, 0)  # (B, G, R)
green_color = (0, 255, 0)  # (B, G, R)
blaсk_color = (0, 0, 0)

area1 = np.array([[0, 0],[frame_width, 0],
                  [frame_width, frame_height // 2],[0, frame_height // 2]], np.int32)

area2 = np.array([[0, frame_height // 2],[frame_width, frame_height // 2],
                  [frame_width, frame_height],[0, frame_height]], np.int32)
ent = 0
ext = 0
obj = {}

**Perform object detection and tracking using the model**

In [ ]:
# Load a model YOLO
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

start_time = time.perf_counter()

while cap.isOpened():
    # Reading video frames
    ret, frame = cap.read()

    if ret:
        # detect objects in image using a model YOLO
        results = model(frame, classes=0, conf=0.6)
        # results = model(frame, classes=(1, 2 ,3, 5, 7), conf=0.8)

        # parsing model parameters
        result = results[0]
        boxes = result.boxes  # Boxes object for bbox outputs

        # Confidence object
        probs = result.probs
        conf = boxes.conf

        # Coordinates of the object's bounding box
        xyxy = boxes.xyxy
        xywh = boxes.xywh

        # Object label (class name)
        cls = boxes.cls.tolist()  # Convert tensor to list
        # for class_index in cls:
        #     class_name = class_names[int(class_index)]
        #     # print("Class:", class_name)

        # #class name
        # class_name = [result.names[box.cls.item()] for box in boxes] # box with cls format, (1, N)
        # print("Class name -->", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)

        # object tracking DeepSORT
        tracks = tracker.update(bboxes_xywh, conf, frame)

        # parsing tracking parameters
        for track in tracker.tracker.tracks:

            # get track_id object
            track_id = track.track_id
            hits = track.hits
            obj.setdefault(track_id, 0)

            # Get bounding box coordinates in (x1, y1, x2, y2) format
            x1, y1, x2, y2 = track.to_tlbr()
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height
            x_obj = int(x1 + w/2)
            y_obj = int(y1 + 0)

            # Counting object
            res_areal1, res_areal2 = -1, -1
            if (frame_height // 2) >= y_obj:
                res_areal1 = 1
            else:
                res_areal2 = 1

            # res_areal1 = cv2.pointPolygonTest(area1, ((x_obj , y_obj)), False )
            # res_areal2 = cv2.pointPolygonTest(area2, ((x_obj , y_obj)), False )

            if obj[track_id] == 1:
                if res_areal2 == 1:
                    obj[track_id] = 2
                    ent += 1
            elif obj[track_id] == 2:
                if res_areal1 == 1:
                    obj[track_id] = 1
                    ext += 1
            elif obj[track_id] == 0:
                if res_areal1 == 1:
                    obj[track_id] = 1
                if res_areal2 == 1:
                    obj[track_id] = 2

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            # display bounding box and text
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            cv2.putText(frame, f"{track_id}", (int(x1) + 10, int(y1) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, blaсk_color, 1, cv2.LINE_AA)

            cv2.circle(frame,(x_obj, y_obj ), 3, blaсk_color,3)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time
        print('fps', fps)

        # display object count and dividing line
        cv2.line(frame, (0, frame_height // 2), (frame_width, frame_height // 2), red_color, thickness=2)
        # cv2.polylines(frame, [area1], True, red_color, 2)
        # cv2.polylines(frame, [area2], True, red_color, 2)

        cv2.putText(frame, f'{ent}', (20, 40), cv2.FONT_HERSHEY_TRIPLEX, 1.0, blaсk_color, 3)
        cv2.putText(frame, f'{ext}', (80, 40), cv2.FONT_HERSHEY_TRIPLEX, 1.0, blaсk_color, 3)

        # Write the frame to the output video file
        out.write(frame)

        # # Show the frame
        # cv2.imshow("Video", og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 10 persons, 66.9ms
Speed: 1.9ms preprocess, 66.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 6.8ms
Speed: 3.9ms preprocess, 6.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 9.1ms
Speed: 5.0ms preprocess, 9.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 7.1ms
Speed: 2.3ms preprocess, 7.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)



fps 0
fps 0
fps 0
fps 0


0: 384x640 10 persons, 10.4ms
Speed: 3.0ms preprocess, 10.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 8.4ms
Speed: 3.0ms preprocess, 8.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 8.8ms
Speed: 4.9ms preprocess, 8.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 8.5ms
Speed: 3.2ms preprocess, 8.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)



fps 0
fps 0
fps 0
fps 7.621968691365138


0: 384x640 10 persons, 10.5ms
Speed: 3.1ms preprocess, 10.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 8.5ms
Speed: 4.3ms preprocess, 8.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 8.0ms
Speed: 4.5ms preprocess, 8.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 8.3ms
Speed: 3.8ms preprocess, 8.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)



...

fps 10.542653664192104
fps 10.542653664192104
fps 10.542653664192104


0: 384x640 7 persons, 15.5ms
Speed: 5.7ms preprocess, 15.5ms inference, 8.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 17.1ms
Speed: 4.6ms preprocess, 17.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 17.6ms
Speed: 5.2ms preprocess, 17.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 9.1ms
Speed: 2.6ms preprocess, 9.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


fps 10.542653664192104
fps 10.542653664192104
fps 13.333875476011398
fps 13.333875476011398



0: 384x640 5 persons, 8.9ms
Speed: 2.5ms preprocess, 8.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)


KeyboardInterrupt: ignored

In [ ]:
# load file mp4
from IPython.display import HTML
from base64 import b64encode
output_path = '/content/Tracking-and-counting-Using-YOLOv8-and-DeepSORT/output.mp4'
# output_path = '/content/test3.mp4'
mp4 = open(output_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
# video
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)
